In [3]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())
print(data)

Data(x=[3, 1], edge_index=[2, 4])


In [4]:
data.validate(raise_on_error=True)

True

In [5]:
data.keys()

['x', 'edge_index']

In [6]:
data['x']

tensor([[-1.],
        [ 0.],
        [ 1.]])

In [8]:
data.num_nodes

3

In [9]:
data.num_edges

4

In [10]:
data.num_node_features

1

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

In [22]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [23]:
len(dataset)

1

In [24]:
dataset.num_classes

7

In [25]:
dataset.num_node_features

1433

In [27]:
data = dataset[0]
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [28]:
data.is_undirected()

True

In [29]:
data.train_mask.sum().item()

140

In [43]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [44]:
len(dataset)

1

In [45]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [46]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        print("Input shape:", x.shape)  # 입력 차원

        x = self.conv1(x, edge_index)
        print("After conv1:", x.shape)  # conv1 이후

        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv2(x, edge_index)
        print("After conv2:", x.shape)  # conv2 이후

        out = F.log_softmax(x, dim=1)
        print("Final output shape:", out.shape)  # 출력 차원
        return out

In [56]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

Input shape: torch.Size([2708, 1433])
After conv1: torch.Size([2708, 16])
After conv2: torch.Size([2708, 7])
Final output shape: torch.Size([2708, 7])
Input shape: torch.Size([2708, 1433])
After conv1: torch.Size([2708, 16])
After conv2: torch.Size([2708, 7])
Final output shape: torch.Size([2708, 7])
Input shape: torch.Size([2708, 1433])
After conv1: torch.Size([2708, 16])
After conv2: torch.Size([2708, 7])
Final output shape: torch.Size([2708, 7])
Input shape: torch.Size([2708, 1433])
After conv1: torch.Size([2708, 16])
After conv2: torch.Size([2708, 7])
Final output shape: torch.Size([2708, 7])
Input shape: torch.Size([2708, 1433])
After conv1: torch.Size([2708, 16])
After conv2: torch.Size([2708, 7])
Final output shape: torch.Size([2708, 7])
Input shape: torch.Size([2708, 1433])
After conv1: torch.Size([2708, 16])
After conv2: torch.Size([2708, 7])
Final output shape: torch.Size([2708, 7])
Input shape: torch.Size([2708, 1433])
After conv1: torch.Size([2708, 16])
After conv2: torch.S

In [55]:
dataset.num_classes

7

In [49]:
model

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

In [47]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

class HierarchicalAttention(nn.Module):
    """계층적 관계에 대한 어텐션 메커니즘"""
    def __init__(self, hidden_dim):
        super(HierarchicalAttention, self).__init__()
        self.attn = nn.Linear(hidden_dim * 2, 1)
        
    def forward(self, parent_embeds, child_embeds):
        """
        부모-자식 노드 간 어텐션 계산
        
        Args:
            parent_embeds: 부모 노드 임베딩 [n_parents, hidden_dim]
            child_embeds: 자식 노드 임베딩 [n_children, hidden_dim]
            
        Returns:
            weighted_embeds: 어텐션 가중치가 적용된 임베딩
        """
        n_parents = parent_embeds.size(0)
        n_children = child_embeds.size(0)
        
        # 각 부모-자식 쌍에 대한 어텐션 점수 계산
        parent_expanded = parent_embeds.unsqueeze(1).expand(n_parents, n_children, -1)
        child_expanded = child_embeds.unsqueeze(0).expand(n_parents, n_children, -1)
        
        # 연결 및 어텐션 점수 계산
        concat = torch.cat([parent_expanded, child_expanded], dim=2)
        attn_scores = self.attn(concat.view(-1, concat.size(2))).view(n_parents, n_children)
        attn_weights = F.softmax(attn_scores, dim=1)
        
        # 가중 합 계산
        weighted_embeds = torch.bmm(attn_weights.unsqueeze(1), child_embeds.unsqueeze(0).expand(n_parents, n_children, -1))
        
        return weighted_embeds.squeeze(1)

class TemporalAttentionLayer(nn.Module):
    """시간적 어텐션 레이어"""
    def __init__(self, input_dim, hidden_dim, num_heads=4):
        super(TemporalAttentionLayer, self).__init__()
        self.attn = nn.MultiheadAttention(hidden_dim, num_heads)
        self.fc = nn.Linear(input_dim, hidden_dim)
        
    def forward(self, x):
        """
        시간적 어텐션 적용
        
        Args:
            x: 입력 시퀀스 [batch_size, seq_len, input_dim]
            
        Returns:
            attn_output: 어텐션이 적용된 시퀀스
        """
        # 차원 변환: [batch_size, seq_len, input_dim] -> [seq_len, batch_size, hidden_dim]
        x = self.fc(x)
        x = x.permute(1, 0, 2)
        
        # 멀티헤드 어텐션 적용
        attn_output, _ = self.attn(x, x, x)
        
        # 차원 복원: [seq_len, batch_size, hidden_dim] -> [batch_size, seq_len, hidden_dim]
        attn_output = attn_output.permute(1, 0, 2)
        
        return attn_output

class SpatioTemporalGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_relations, num_layers=2):
        super(SpatioTemporalGNN, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_relations = num_relations
        self.num_layers = num_layers
        
        # 입력 변환 레이어
        self.input_fc = nn.Linear(input_dim, hidden_dim)
        
        # RGCN 레이어 (계층적 관계 모델링)
        self.rgcn_layers = nn.ModuleList([
            RGCNConv(hidden_dim, hidden_dim, num_relations)
            for _ in range(num_layers)
        ])
        
        # GAT 레이어 (노드 간 영향력 파악)
        self.gat_layers = nn.ModuleList([
            GATConv(hidden_dim, hidden_dim)
            for _ in range(num_layers)
        ])
        
        # 시간적 어텐션 레이어
        self.temporal_attn = TemporalAttentionLayer(hidden_dim, hidden_dim)
        
        # 계층적 어텐션 레이어
        self.hierarchical_attn = HierarchicalAttention(hidden_dim)
        
        # 출력 변환 레이어
        self.output_fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, data):
        x, edge_index, edge_type = data.x, data.edge_index, data.edge_type
        batch_size, num_nodes, seq_len = x.size(0), x.size(1), x.size(2)
        
        # 입력 변환: [batch_size, num_nodes, seq_len] -> [batch_size, num_nodes, hidden_dim]
        x = self.input_fc(x.view(batch_size * num_nodes, seq_len)).view(batch_size, num_nodes, self.hidden_dim)
        
        # 시간적 어텐션 적용
        x = self.temporal_attn(x)
        
        # 그래프 컨볼루션 적용 (각 시간 단계에 대해)
        for i in range(self.num_layers):
            # RGCN: 계층적 관계 모델링
            x_rgcn = x.view(-1, self.hidden_dim)
            x_rgcn = self.rgcn_layers[i](x_rgcn, edge_index, edge_type)
            x_rgcn = x_rgcn.view(batch_size, num_nodes, self.hidden_dim)
            
            # GAT: 노드 간 영향력 파악
            x_gat = x.view(-1, self.hidden_dim)
            x_gat = self.gat_layers[i](x_gat, edge_index)
            x_gat = x_gat.view(batch_size, num_nodes, self.hidden_dim)
            
            # 결합 및 비선형성 적용
            x = F.relu(x_rgcn + x_gat)
            x = F.dropout(x, p=0.1, training=self.training)
        
        # 출력 변환: [batch_size, num_nodes, hidden_dim] -> [batch_size, num_nodes, output_dim]
        x = self.output_fc(x.view(batch_size * num_nodes, self.hidden_dim)).view(batch_size, num_nodes, self.output_dim)
        
        return x

ModuleNotFoundError: No module named 'sklearn'

In [2]:
learning_rate = 1e-3
num_epochs = 10

# 모델 초기화
input_dim = 28  # 28일 데이터 (조정 필요)
hidden_dim = 64
output_dim = 28  # 28일 예측
num_relations = 4  # geo_hierarchy, prod_hierarchy, agg_hierarchy, cross_hierarchy

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = SpatioTemporalGNN(input_dim, hidden_dim, output_dim, num_relations)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0

    for i

    model.train()
    
    
    # 데이터 로딩 (예시)
    # data = load_data()  # 사용자 정의 데이터 로딩 함수
    # data = data.to(device)
    
    output = model(data)
    
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')